In [ ]:
!python --version

In [ ]:
import langchain
langchain.__version__


In [ ]:
### Initialize langchain agents and tools
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

from dotenv import load_dotenv
load_dotenv()
import os
open_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
!pip install -U duckduckgo-search

In [ ]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
search_tool = Tool(
    name= "DuckDuckGo Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to perform web searches such as latest news, entertainement movies etc using DuckDuckGo"
)
llm = OpenAI(temperature =0.5)




In [ ]:
!pip install wikipedia

In [ ]:
## Initialize the Wikipedia 
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
wikipedia_tool= Tool(
    name= "Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to perform wikipedia searches for information "
)

tools=[wikipedia_tool]


In [ ]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///example.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sqllite_tool = toolkit.get_tools()
tools = sqllite_tool


In [ ]:
agent = initialize_agent(tools = tools, 
                         llm=llm,
                         agent="zero-shot-react-description",
                        verbose=True
                        )
query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("response: ", response1)

In [ ]:
!pip install arxiv

In [ ]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain.agents import load_tools
arxiv_tool = Tool(
    name="Arxiv Search",
    func= ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)





In [ ]:
##tools = load_tools(["arxiv_tool", "wikipedia_tool", "search_tool", "sqllite_tool"], llm=llm)
tools = [arxiv_tool, search_tool]

In [ ]:
agent = initialize_agent(tools, 
                         llm,
                         agent="zero-shot-react-description",
                        verbose=True,
                        handle_parsing_errors=True
                        )


In [ ]:
##query1 = "Search for research papers on Diffusion Models in 2024."
##query1 = "Find recent papers on Generative AI and what is the latest movie trending in Charlotte USA this week"
query1 = "In which city was Facebook office started and What are some of the recent papers on Generative AI"
response1 = agent.run(query1)
print("response: ", response1)